In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import yaml
from pathlib import Path

import pandas as pd
import xarray as xr
import pandas_indexing.accessors
from aneris.harmonize import Harmonizer
from aneris.downscaling import Downscaler
from aneris.grid import Gridder
from pandas import DataFrame
from pandas_indexing import isin, semijoin, concat, ismatch
from IPython import get_ipython

from concordia import VariableDefinitions, RegionMapping, combine_countries, CondordiaMagics
from pandas_indexing.units import set_openscm_registry_as_default

In [3]:
get_ipython().register_magics(CondordiaMagics)

In [4]:
ur = set_openscm_registry_as_default()

# Set which parts of the workflow you would like to execute

In [5]:
execute_harmonization = True
execute_downscaling = True
execute_gridding = True

# Read model and historic data including overrides

To run this code, create a file called `config.yaml` in this directory pointing to the correct data file locations, e.g.,

```
# config.yaml
base_path: "/Users/{macuser}/Library/CloudStorage/OneDrive-SharedLibraries-IIASA/RESCUE - WP 1/data"
data_path: "../data"
country_combinations:
  sdn_ssd: ["ssd", "sdn"]
  isr_pse: ["isr", "pse"]
  srb_ksv": ["srb", "srb (kosovo)"]
```


In [6]:
with open("config.yaml", "r") as stream:
    config = yaml.safe_load(stream)

In [7]:
base_path = Path(config["base_path"])
data_path = Path(config["data_path"])
out_path = base_path.parent / "analysis" / "harmonization"

base_year = 2020  # in which year scenario data should be harmonized to historical data
country_combinations = config["country_combinations"]

## Variable definition files

The variable definition file is a CSV or yaml file that needs to contain the `variable`-name, its `sector`, `gas` components and whether it is expected `global` (or regional instead).

Here we generate one based on the cmip6 historical data we have that could be used as a basis but we would want to finetune this by hand.


In [8]:
variabledefs = VariableDefinitions.from_csv(data_path / "variabledefs-rescue.csv")
variabledefs.data.tail()

,,,unit,global,has_history
variable,gas,sector,,,
CEDS+|9+ Sectors|Emissions|HFC,HFC,Total,Mt CO2eq/yr,True,True
CEDS+|9+ Sectors|Emissions|C2F6,C2F6,Total,kt C2F6/yr,True,True
CEDS+|9+ Sectors|Emissions|CF4,CF4,Total,kt CF4/yr,True,True
CEDS+|9+ Sectors|Emissions|SF6,SF6,Total,kt SF6/yr,True,True
CEDS+|9+ Sectors|Emissions|N2O,N2O,Total,kt N2O/yr,True,True


## RegionMapping helps reading in a region definition file


In [9]:
regionmapping = RegionMapping.from_regiondef(
    base_path / "iam_files/rescue/regionmappingH12.csv",
    country_column="CountryCode",
    region_column="RegionCode",
    sep=";",
)
regionmapping.data = combine_countries(
    regionmapping.data, **country_combinations, agg_func="last"
)

In [10]:
regionmapping.data.unique()

array(['LAM', 'OAS', 'SSA', 'EUR', 'NEU', 'MEA', 'REF', 'CAZ', 'CHA',
       'IND', 'JPN', 'USA'], dtype=object)

## Model and historic data read in

Can be read in and prepared using `read_iamc` or the `variabledefs`


In [11]:
hist_ceds = (
    pd.read_csv(
        base_path / "historical/rescue/ceds_2017_extended.csv", index_col=list(range(4))
    )
    .rename(index={"NMVOC": "VOC", "SO2": "Sulfur"}, level="gas")
    .rename(index={"Mt NMVOC/yr": "Mt VOC/yr"}, level="unit")
    .rename(columns=int)
    .pix.format(variable="CEDS+|9+ Sectors|Emissions|{gas}|{sector}", drop=True)
    .pix.assign(model="History", scenario="CEDS")
)

In [12]:
hist_global = (
    pd.read_excel(base_path / "historical/rescue/global_trajectories.xlsx", index_col=list(range(5)))
    .rename_axis(index=str.lower)
    .rename_axis(index={"region": "country"})
    .rename(index=lambda s: s.removesuffix("|Unharmonized"), level="variable")
)

In [13]:
hist_gfed = pd.read_csv(
    base_path / "historical/rescue/gfed/GFED2015_extended.csv", index_col=list(range(5))
).rename(columns=int)

In [14]:
hist = (
    concat([hist_ceds, hist_global, hist_gfed]).droplevel(["model", "scenario"])
    .pipe(combine_countries, **country_combinations)
    .pipe(
        variabledefs.load_data,
        extend_missing=True,
        levels=["country", "gas", "sector", "unit"],
    )
)
hist.head()

,,,year,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2020
country,gas,sector,unit,,,,,,,,,,,,,,,,,,,,,
World,BC,Agricultural Waste Burning,Mt BC/yr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.196335,0.209205,2.086170e-01,2.007464e-01,1.892032e-01,1.830717e-01,1.664993e-01,1.660367e-01,1.673890e-01,1.729010e-01
abw,BC,Agricultural Waste Burning,Mt BC/yr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,4.100000e-09,4.100000e-09,4.100000e-09,2.152000e-08,2.152000e-08,1.742000e-08,1.957200e-08,2.946408e-08
afg,BC,Agricultural Waste Burning,Mt BC/yr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000003,0.000003,2.599827e-06,3.375778e-06,3.195890e-06,2.889030e-06,2.594670e-06,2.603757e-06,1.868047e-06,2.258981e-06
ago,BC,Agricultural Waste Burning,Mt BC/yr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003883,0.003675,3.273211e-03,2.940681e-03,2.722290e-03,2.646585e-03,2.726854e-03,2.762196e-03,2.805735e-03,3.677709e-03
aia,BC,Agricultural Waste Burning,Mt BC/yr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [15]:
with ur.context("AR4GWP100"):
    model = (
        pd.read_csv(
            base_path / "iam_files/rescue/REMIND-MAgPIE-CEDS-RESCUE-Tier1-Extension-2023-07-27.csv",
            index_col=list(range(5)),
            sep=";",
        )
        .drop(["Unnamed: 21"], axis=1)
        .rename(
            index={"Mt CO2-equiv/yr": "Mt CO2eq/yr", "Mt NOX/yr": "Mt NOx/yr", "kt HFC134a-equiv/yr": "kt HFC134a/yr"},
            level="Unit",
        )
        .pix.convert_unit({"kt HFC134a/yr": "Mt CO2eq/yr"}, level="Unit")
        .rename(
            index=lambda s: s.removesuffix("|Total"),
            level="Variable"
        )
        .pipe(
            variabledefs.load_data,
            extend_missing=True,
            levels=["model", "scenario", "region", "gas", "sector", "unit"],
        )
    )
model.pix

Index:
 * model    : REMIND-MAgPIE 3.2-4.6 (1)
 * scenario : RESCUE-Tier1-Extension-2023-07-27-Baseline, ... (19)
 * region   : CAZ, CHA, EUR, IND, JPN, LAM, MEA, NEU, OAS, REF, ... World (13)
 * gas      : BC, C2F6, CF4, CH4, CO2, CO, HFC, N2O, NH3, NOx, OC, ... VOC (14)
 * sector   : Agricultural Waste Burning, Agriculture, Aircraft, ... Peat Burning (22)
 * unit     : Mt BC/yr, kt C2F6/yr, kt CF4/yr, Mt CH4/yr, ... Mt VOC/yr (14)

Columns:
 * year : 2005, 2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, ... 2100 (16)

In [16]:
harm_overrides = pd.read_excel(base_path / "iam_files" / "rescue" / "harmonization_overrides.xlsx", index_col=list(range(3))).method
harm_overrides

gas  sector                      region
BC   Industrial Sector           SSA       constant_offset
CH4  Forest Burning              MEA        constant_ratio
                                 JPN        constant_ratio
NH3  Agricultural Waste Burning  LAM        constant_ratio
                                 NEU        constant_ratio
OC   Industrial Sector           SSA       constant_offset
VOC  Industrial Sector           OAS       constant_offset
                                 SSA       constant_offset
Name: method, dtype: object

In [17]:
hist_available = hist.pix.unique(["gas", "sector"])

In [18]:
model.pix.unique(["gas", "sector"]).difference(hist_available)

MultiIndex([], names=['gas', 'sector'])

# Harmonization

## Preparation of input data


In [19]:
hist_agg = pd.concat(
    [
        hist.pix.semijoin(variabledefs.index_regional, how="inner").pipe(
            regionmapping.aggregate, dropna=True
        ),
        hist.pix.semijoin(variabledefs.index_global, how="inner")
        .loc[isin(country="World")]
        .rename_axis(index={"country": "region"}),
    ]
)

In [20]:
model_agg = pd.concat(
    [
        model.pix.semijoin(variabledefs.index_regional, how="inner").loc[
            isin(region=regionmapping.data.unique())
        ],
        model.pix.semijoin(variabledefs.index_global, how="inner").loc[
            isin(region="World")
        ],
    ]
).pix.semijoin(hist_agg.index, how="inner")

## Harmonize all model, scenarios combinations


In [21]:
luc_sectors = ['Agricultural Waste Burning', 'Grassland Burning', 'Forest Burning']

In [22]:
def harmonize(model_agg, hist_agg, config, overrides):
    harmonized = []
    for m, s in model_agg.index.pix.unique(["model", "scenario"]):
        scen = model_agg.loc[isin(model=m, scenario=s)].droplevel(["model", "scenario"])
        h = Harmonizer(
            scen,
            hist_agg.pix.semijoin(scen.index, how="right").loc[:, 2000:],
            harm_idx=scen.index.names,
            config=config,
        )
        result = h.harmonize(
            year=base_year, overrides=None if overrides.empty else overrides
        ).sort_index()
        methods = h.methods(year=base_year)
        result = result.pix.assign(
            method=methods.pix.semijoin(result.index, how="right")
        )
        harmonized.append(result.pix.assign(model=m, scenario=s))
    harmonized = pd.concat(harmonized)

    return harmonized

In [23]:
version = "2023-08-09"
harmonized_path = out_path / f"harmonized-only-{version}.csv"

In [24]:
%%execute_or_lazy_load execute_harmonization harmonized = pd.read_csv(harmonized_path)
is_luc = isin(sector=luc_sectors)
harmonized = concat(
    [
        harmonize(
            model_agg.loc[is_luc],
            hist_agg.loc[is_luc],
            config=dict(),
            overrides=harm_overrides.loc[is_luc],
        ),
        harmonize(
            model_agg.loc[~is_luc],
            hist_agg.loc[~is_luc],
            config=dict(default_luc_method="reduce_ratio_2080"),
            overrides=harm_overrides.loc[~is_luc],
        ),
    ]
)
harmonized.to_csv(harmonized_path)

INFO:root:Harmonizing with reduce_offset_2150_cov
INFO:root:Harmonizing with constant_ratio
INFO:root:Harmonizing with reduce_ratio_2080
INFO:root:Harmonizing with model_zero


INFO:root:Harmonizing with reduce_offset_2150_cov
INFO:root:Harmonizing with constant_ratio
INFO:root:Harmonizing with reduce_ratio_2080
INFO:root:Harmonizing with model_zero
INFO:root:Harmonizing with reduce_offset_2150_cov
INFO:root:Harmonizing with constant_ratio
INFO:root:Harmonizing with reduce_ratio_2080
INFO:root:Harmonizing with model_zero
INFO:root:Harmonizing with reduce_offset_2150_cov
INFO:root:Harmonizing with constant_ratio
INFO:root:Harmonizing with reduce_ratio_2080
INFO:root:Harmonizing with model_zero
INFO:root:Harmonizing with reduce_offset_2150_cov
INFO:root:Harmonizing with constant_ratio
INFO:root:Harmonizing with reduce_ratio_2080
INFO:root:Harmonizing with model_zero
INFO:root:Harmonizing with reduce_offset_2150_cov
INFO:root:Harmonizing with constant_ratio
INFO:root:Harmonizing with reduce_ratio_2080
INFO:root:Harmonizing with model_zero
INFO:root:Harmonizing with reduce_offset_2150_cov
INFO:root:Harmonizing with constant_ratio
INFO:root:Harmonizing with reduce

In [25]:
harmonized.loc[(harmonized < 0).any(axis=1)].loc[~ismatch(sector="CDR*")]

,,,,,,year,2020,2025,2030,2035,2040,2045,2050,2055,2060,2070,2080,2090,2100
gas,sector,region,unit,method,model,scenario,,,,,,,,,,,,,


In [26]:
def make_totals(df):
    original_levels = df.index.names
    if "method" in original_levels:  # need to process harm
        df = df.droplevel("method")
    level = df.index.names
    ret = concat(
        [
            (
                df.loc[~isin(region="World")]  # don't count aviation
                .groupby(level.difference(["region"]))
                .agg(lambda s: s.sum(skipna=False))
                .pix.assign(region="World", order=level)
            ),
            (
                df.loc[~(isin(sector="Total") | isin(region="World"))]
                .groupby(level.difference(["sector"]))
                .agg(lambda s: s.sum(skipna=False))
                .pix.assign(sector="Total", order=level)
            ),
            (
                df.loc[~isin(sector="Total")]  # don't count global totals
                .groupby(level.difference(["region", "sector"]))
                .agg(lambda s: s.sum(skipna=False))
                .pix.assign(region="World", sector="Total", order=level)
            ),
        ]
    )
    if "method" in original_levels:  # need to process harm
        ret = ret.pix.assign(method="aggregate", order=original_levels)
    return ret

model_agg = concat([model_agg, make_totals(model_agg)])
hist_agg = concat([hist_agg, make_totals(hist_agg)])
harmonized = concat([harmonized, make_totals(harmonized)])

In [27]:
data = concat(
    [
        model_agg.pix.format(
            variable="Emissions|{gas}|{sector}|Unharmonized", drop=True
        ),
        harmonized.pix.format(
            variable="Emissions|{gas}|{sector}|Harmonized|{method}", drop=True
        ),
        hist_agg.loc[:, 1990:].pix.format(
            model="Historic",
            scenario="Synthetic (GFED/CEDS/Global)",
            variable="Emissions|{gas}|{sector}",
            drop=True,
        ),
    ],
    order=["model", "scenario", "region", "variable", "unit"],
).sort_index(axis=1)
data.to_csv(out_path / f"harmonization-{version}.csv")

In [28]:
hfc_distribution = (
    pd.read_csv(
        base_path
        / "harmonization_postprocessing"
        / "rescue"
        / "rescue_hfc_scenario.csv",
        index_col=0,
    )
    .rename_axis("hfc")
    .rename(columns=int)
)

def split_hfc(df):
    return concat(
        [
            df.loc[~isin(gas="HFC")],
            df.pix.multiply(hfc_distribution.pix.assign(gas="HFC"), join="inner")
            .droplevel("gas")
            .rename_axis(index={"hfc": "gas"}),
        ]
    )
data = concat(
    [
        split_hfc(model_agg).pix.format(
            variable="Emissions|{gas}|{sector}|Unharmonized", drop=True
        ),
        split_hfc(harmonized).pix.format(
            variable="Emissions|{gas}|{sector}|Harmonized|{method}", drop=True
        ),
        split_hfc(hist_agg.loc[:, 1990:]).pix.format(
            model="Historic",
            scenario="Synthetic (GFED/CEDS/Global)",
            variable="Emissions|{gas}|{sector}",
            drop=True,
        ),
    ],
    order=["model", "scenario", "region", "variable", "unit"],
).sort_index(axis=1)
data.to_csv(out_path / f"harmonization-{version}-splithfc.csv")

# Downscaling


## Prepare GDP proxy

Read in different GDP scenarios for SSP1 to SSP5 from SSP DB.


In [29]:
gdp = (
    pd.read_csv(
        base_path / "historical" / "SspDb_country_data_2013-06-12.csv",
        index_col=list(range(5)),
    )
    .rename_axis(index=str.lower)
    .loc[
        isin(
            model="OECD Env-Growth",
            scenario=[f"SSP{n+1}_v9_130325" for n in range(5)],
            variable="GDP|PPP",
        )
    ]
    .dropna(how="all", axis=1)
    .rename_axis(index={"scenario": "ssp", "region": "country"})
    .rename(index=str.lower, level="country")
    .rename(columns=int)
    .pix.project(["ssp", "country"])
    .pipe(combine_countries, **country_combinations)
)
gdp.head()

2000     2005     2010     2015     2020     2025   
ssp            country                                                         
SSP1_v9_130325 abw        0.000    0.128    0.151    0.205    0.266    0.357  \
               afg        0.000   22.372   37.237   47.643   59.153   78.506   
               ago       34.484   55.315   98.686  127.295  171.984  203.363   
               alb       14.743   19.170   24.545   26.919   30.452   35.126   
               are      209.548  272.055  318.142  439.900  536.362  652.339   

                           2030     2035      2040      2045  ...      2055   
ssp            country                                        ...             
SSP1_v9_130325 abw        0.492    0.679     0.926     1.237  ...     2.067  \
               afg      109.567  157.250   227.005   324.841  ...   627.328   
               ago      232.320  261.469   304.102   369.997  ...   591.431   
               alb       41.037   48.034    55.608    63.289  ...    76.939   
               are      787.776  927.181  1061.287  1176.838  ...  1343.919   

                            2060      2065      2070      2075      2080   
ssp            country                                                     
SSP1_v9_130325 abw         2.586     3.165     3.797     4.470     5.171  \
               afg       839.381  1094.144  1389.577  1722.005  2086.194   
               ago       750.860   942.049  1160.284  1400.116  1655.046   
               alb        83.030    88.331    92.473    95.687    98.004   
               are      1421.730  1487.358  1535.672  1560.609  1569.904   

                            2085      2090      2095      2100  
ssp            country                                          
SSP1_v9_130325 abw         5.842     6.469     7.034     7.505  
               afg      2474.187  2878.709  3293.204  3707.958  
               ago      1917.629  2181.823  2441.164  2687.836  
               alb        99.153    99.277    98.596    97.006  
               are      1565.244  1550.719  1526.111  1486.250  

[5 rows x 21 columns]

Determine likely SSP for each harmonized pathway from scenario string and create proxy data aligned with pathways


In [30]:
SSP_per_pathway = (
    harmonized.index.pix.project(["model", "scenario"])
    .unique()
    .to_frame()
    .scenario.str.extract("(SSP[1-5])")[0]
    .fillna("SSP2")
)
gdp = semijoin(
    gdp,
    SSP_per_pathway.index.pix.assign(ssp=SSP_per_pathway + "_v9_130325"),
    how="right",
).pix.project(["model", "scenario", "country"])
gdp.head()

2000   
model                 scenario                                   country            
REMIND-MAgPIE 3.2-4.6 RESCUE-Tier1-Extension-2023-07-27-Baseline abw        0.000  \
                                                                 afg        0.000   
                                                                 ago       34.484   
                                                                 alb       14.743   
                                                                 are      209.548   

                                                                             2005   
model                 scenario                                   country            
REMIND-MAgPIE 3.2-4.6 RESCUE-Tier1-Extension-2023-07-27-Baseline abw        0.128  \
                                                                 afg       22.372   
                                                                 ago       55.315   
                                                                 alb       19.170   
                                                                 are      272.055   

                                                                             2010   
model                 scenario                                   country            
REMIND-MAgPIE 3.2-4.6 RESCUE-Tier1-Extension-2023-07-27-Baseline abw        0.151  \
                                                                 afg       37.237   
                                                                 ago       98.686   
                                                                 alb       24.545   
                                                                 are      318.142   

                                                                             2015   
model                 scenario                                   country            
REMIND-MAgPIE 3.2-4.6 RESCUE-Tier1-Extension-2023-07-27-Baseline abw        0.205  \
                                                                 afg       48.185   
                                                                 ago      128.679   
                                                                 alb       26.994   
                                                                 are      441.163   

                                                                             2020   
model                 scenario                                   country            
REMIND-MAgPIE 3.2-4.6 RESCUE-Tier1-Extension-2023-07-27-Baseline abw        0.264  \
                                                                 afg       60.175   
                                                                 ago      172.787   
                                                                 alb       30.562   
                                                                 are      539.009   

                                                                             2025   
model                 scenario                                   country            
REMIND-MAgPIE 3.2-4.6 RESCUE-Tier1-Extension-2023-07-27-Baseline abw        0.342  \
                                                                 afg       77.749   
                                                                 ago      200.062   
                                                                 alb       34.682   
                                                                 are      649.478   

                                                                             2030   
model                 scenario                                   country            
REMIND-MAgPIE 3.2-4.6 RESCUE-Tier1-Extension-2023-07-27-Baseline abw        0.442  \
                                                                 afg      101.896   
                                                                 ago      218.446   
                                                                 alb       3

In [31]:
downscaled_path = out_path / f"downscaled-only-{version}.csv"

The regionmapping has several countries which are not part of the original gdp data, therefore we remove those countries (and effectively split the regional emissions among fewer countries). The missing countries are:

In [32]:
regionmapping.data.index.difference(gdp.pix.unique("country"))

Index(['aia', 'ala', 'and', 'asm', 'ata', 'atf', 'atg', 'bes', 'blm', 'bmu',
       'bvt', 'cck', 'cok', 'cuw', 'cxr', 'cym', 'dma', 'esh', 'flk', 'fro',
       'fsm', 'ggy', 'gib', 'glp', 'grd', 'grl', 'guf', 'gum', 'hmd', 'imn',
       'iot', 'jey', 'kir', 'kna', 'lie', 'maf', 'mco', 'mhl', 'mnp', 'msr',
       'mtq', 'myt', 'nfk', 'niu', 'nru', 'pcn', 'plw', 'prk', 'reu', 'sgs',
       'shn', 'sjm', 'smr', 'spm', 'sxm', 'syc', 'tca', 'tkl', 'tuv', 'umi',
       'vat', 'vgb', 'vir', 'wlf'],
      dtype='object', name='country')

In [33]:
# Remove countries from regionmapping that the GDP proxy does not have
regionmapping = RegionMapping(regionmapping.data.loc[isin(country=gdp.pix.unique("country"))])

In [34]:
%%execute_or_lazy_load execute_downscaling downscaled = pd.read_csv(downscaled_path)
downscaler = Downscaler(
    harmonized.pix.semijoin(variabledefs.index_regional, how="inner").loc[~isin(region="World")].droplevel("method"),
    hist.pix.semijoin(variabledefs.index_regional, how="inner"),
    base_year,
    regionmapping.data,
    gdp=gdp,
)
methods = downscaler.methods()
downscaled = downscaler.downscale(methods).sort_index()
downscaled = downscaled.pix.assign(method=methods.pix.semijoin(downscaled.index, how="right"))
downscaled.to_csv(downscaled_path)

/Users/coroa/repos/aneris/src/aneris/methods.py:389: RuntimeWarning: invalid value encountered in scalar divide
  return np.abs(np.std(x) / np.mean(x))


# Gridding


In [35]:
idxr = xr.open_dataarray(
    base_path / "gridding_process_files" / "iso_mask.nc", chunks={"iso": 20}
).rename({"iso": "country"})

In [36]:
proxy_dir = base_path / "gridding_process_files" / "proxy_rasters"
proxy_cfg = pd.concat(
    [
        # DataFrame(
        #     {
        #         "path": proxy_dir.glob("aircraft_*.nc"),
        #         "name": "em-AIR-anthro",
        #         "separate_shares": False,
        #     }
        # ),
        DataFrame(
            {
                "path": proxy_dir.glob("anthro_*.nc"),
                "name": "em-anthro",
                "separate_shares": False,
            }
        ),
        DataFrame(
            {
                "path": proxy_dir.glob("openburning_*.nc"),
                "name": "em-openburning",
                "separate_shares": True,
            }
        ),
        # DataFrame({"path": proxy_dir.glob("shipping_*.nc"), "name": ..., "template": ..., "separate_shares": False})
    ]
).assign(
    name=lambda df: df.path.map(lambda p: p.stem.split("_")[1]) + "-" + df.name,
    template="{name}_emissions_{model}-{scenario}_201501-210012",
)
proxy_cfg.head()

,path,name,separate_shares,template
0,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,OC-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012
1,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,CO-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012
2,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,NH3-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012
3,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,NOx-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012
4,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,CO2-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012


In [37]:
sector_mapping = {
    "AIR": "Aircraft",
    "SHP": "International Shipping",
    "AWB": "Agricultural Waste Burning",
    "AGR": "Agriculture",
    "ENE": "Energy Sector",
    "FRTB": "Forest Burning",
    "GRSB": "Grassland Burning",
    "IND": "Industrial Sector",
    "PEAT": "Peat Burning",
    "RCO": "Residential Commercial Other",
    "SLV": "Solvents Production and Application",
    "TRA": "Transportation Sector",
    "WST": "Waste",
}

In [38]:
kg_per_mt = 1e9
s_per_yr = 365 * 24 * 60 * 60
downscaled = (
    downscaled.rename(index=lambda s: re.sub("Mt (.*)/yr", r"kg \1/s", s), level="unit")
    * kg_per_mt
    / s_per_yr
)

In [39]:
downscaled = downscaled.droplevel("region")
downscaled.head()

year                                                                                                                                       2020   
gas sector                     unit    model                 scenario                                           country method                    
BC  Agricultural Waste Burning kg BC/s REMIND-MAgPIE 3.2-4.6 RESCUE-Tier1-Extension-2023-07-27-Baseline         aus     ipat_2100_gdp  0.114939  \
                                                                                                                can     ipat_2100_gdp  0.043853   
                                                                                                                nzl     ipat_2100_gdp  0.001236   
                                                             RESCUE-Tier1-Extension-2023-07-27-PkBudg_cp0400... aus     ipat_2100_gdp  0.114939   
                                                                                                                can     ipat_2100_gdp  0.043853   

year                                                                                                                                       2025   
gas sector                     unit    model                 scenario                                           country method                    
BC  Agricultural Waste Burning kg BC/s REMIND-MAgPIE 3.2-4.6 RESCUE-Tier1-Extension-2023-07-27-Baseline         aus     ipat_2100_gdp  0.149088  \
                                                                                                                can     ipat_2100_gdp  0.059367   
                                                                                                                nzl     ipat_2100_gdp  0.001802   
                                                             RESCUE-Tier1-Extension-2023-07-27-PkBudg_cp0400... aus     ipat_2100_gdp  0.164012   
                                                                                                                can     ipat_2100_gdp  0.065310   

year                                                                                                                                       2030   
gas sector                     unit    model                 scenario                                           country method                    
BC  Agricultural Waste Burning kg BC/s REMIND-MAgPIE 3.2-4.6 RESCUE-Tier1-Extension-2023-07-27-Baseline         aus     ipat_2100_gdp  0.180687  \
                                                                                                                can     ipat_2100_gdp  0.076080   
                                                                                                                nzl     ipat_2100_gdp  0.002481   
                                                             RESCUE-Tier1-Extension-2023-07-27-PkBudg_cp0400... aus     ipat_2100_gdp  0.212079   
                                                                                                                can     ipat_2100_gdp  0.089298   

year                                                                                                                                       2035   
gas sector                     unit    model                 scenario                                           country method                    
BC  Agricultural Waste Burning kg BC/s REMIND-MAgPIE 3.2-4.6 RESCUE-Tier1-Extension-2023-07-27-Baseline         aus     ipat_2100_gdp  0.197379  \
                                                                                                                can     ipat_2100_gdp  0.088994   
                                                                                                                nzl     ipat_2100_gdp  0.003127   
                                                             RESCUE-Tier1-Extension-2023-07-27-PkBudg_cp0400... aus     ipat_2100_gdp  0.234611   
                            

In [ ]:
gridder = Gridder(
    downscaled,
    idxr,
    proxy_cfg,
    index_mappings=dict(sector=sector_mapping),
    output_dir="../results",
)

In [ ]:
%%execute_or_lazy_load execute_gridding
gridder.grid(skip_check=True)